### import twitter data

In [23]:
import pandas as pd


In [24]:
data = pd.read_csv("gender_classifier.csv", encoding = "latin1")

In [25]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [26]:
data = pd.concat([data.gender, data.description], axis = 1)

- Eksik verileri düzenler (None olanları çıkardım)

In [27]:
data.dropna(axis = 0, inplace = True)

In [28]:
data.gender = [1 if each == "female" else 0 for each in data.gender]

### Textleri cleaning etme (Cleaning data)

- Regular expression (RE)

In [29]:
import re

In [30]:
first_description = data.description[4]

- a dan z ye ve A dan Z ye kadar olan harfleri bulur, geri kalanları " " (boşluk) ile yer değiştir  

In [31]:
description = re.sub(r"[^a-zA-Z]", " ", first_description)

- Tüm harfleri küçük harf yapar.

In [32]:
description = description.lower()

### Stopwords (irrelavant words)

- Mesela ingilizcedeki "the", "a", "an", "and" vs kelimeleri ortadan kaldırır. Çünkü bu kelimeler sınıflandırma yaparken tam olarak ayaırt edemeyiz  

In [33]:
import nltk as nlp
nlp.download('punkt')
nlp.download('wordnet')
nlp.download("stopwords") # Corpus diye klasörün içine indirir.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
from nltk.corpus import stopwords # Sonra corpus klasöründen import ediyorum 

- Split veya tokenizer kullanılır.
- description = description.split() --> split için

In [35]:
# tokenizer ederim
description = nlp.word_tokenize(description)

- split kullanırsak "shouldn't" gibi kelimeler "should" ve "n't" olarak ayrılmaz
- Tokenizer kullanırsak  ayrılır

- Gereksiz kelimeleri çıkarma

In [36]:
description = [word for word in description if not word in set(description)]

### Lemmatization

- lemmatization ile köklerini buluruz. 
- gitmeyeceğim -> git

In [37]:
lemma = nlp.WordNetLemmatizer()
description = [lemma.lemmatize(word) for word in description]

In [38]:
description = " ".join(description)

## - Tüm Veri İçin Yapma

In [39]:
description_list = []

for description in data.description:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   # buyuk harftan kucuk harfe cevirme
    description = nlp.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)

### Bag of Word

In [40]:
from sklearn.feature_extraction.text import CountVectorizer # bag of word yaratmak için kullandığım method

max_features = 5000 # en çok kullandığım kelimelerin içinden 5000 tanesini seç
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
# CountVectorizer methodun içine lowercase ve tokenizer da kullanabilirdik.

sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()

### Text Classification

In [41]:
x = sparce_matrix
y = data.iloc[:,0].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 3)

In [42]:
x.shape

(16224, 5000)

In [43]:
y.shape

(16224,)

- Naive Bayes (GaussionNB)

In [44]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)

y_predict = nb.predict(x_test)
print("accuracy: ", nb.score(y_predict.reshape(-1, 1), y_test))

accuracy:  0.4548536209553159
